In [4]:
#Classification method
#predict classes, the probability that a specific entry, its within one or more class

In [ ]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook

In [5]:
from __future__ import absolute_import, division, print_function, unicode_literals


import tensorflow as tf

import pandas as pd

In [6]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']
# Lets define some constants to help us later on

In [ ]:
#load data set train/test
train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)
# Here we use keras (a module inside of TensorFlow) to grab our datasets and read them into a pandas dataframe

In [ ]:
train.head() #dont need to encode the no numerical data (species)

In [ ]:
train_y = train.pop('Species')
test_y = test.pop('Species')
train.head() # the species column is now gone

In [18]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


In [19]:
#input function
#dont have any epochs and the batch size is diferent
def input_fn(features, labels, training=True, batch_size=256):
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

In [20]:
# Feature columns describe how to use the input.
my_feature_columns = []
for key in train.keys(): #loop through the columns 
    my_feature_columns.append(tf.feature_column.numeric_column(key=key)) #get the feature columns
print(my_feature_columns)

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [21]:
# Have diferent classification models
# Dnn = Deep Neural Network
# Build a DNN with 2 hidden layers with 30 and 10 hidden nodes each.
# Define the classifier
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 30 and 10 nodes respectively.
    hidden_units=[30, 10],
    # The model must choose between 3 classes.
    n_classes=3)

In [ ]:
classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True), #this is a one line function
    steps=5000) #set amount of step to go through
# We include a lambda to avoid creating an inner function previously

In [23]:
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(test, test_y, training=False))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))


Test set accuracy: 0.800



In [ ]:
def input_fn(features, batch_size=256): #the user can write the features numbers and the classificator its going to predict the flower class
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
predict = {}

print("Please type numeric values as prompted.")
for feature in features:
  valid = True
  while valid: 
    val = input(feature + ": ")
    if not val.isdigit(): valid = False

  predict[feature] = [float(val)] #create a dictionary with the numbers and their keys
  
predictions = classifier.predict(input_fn=lambda: input_fn(predict)) #dictionary like parameter
for pred_dict in predictions:
    #print(pred_dict)
    class_id = pred_dict['class_ids'][0] #'class_ids': array([2])
    #print(class_id)
    probability = pred_dict['probabilities'][class_id] #'probabilities': array([0.24913281, 0.29840103, 0.45246622]
    
    print('Prediction is "{}" ({:.1f}%)'.format(
        SPECIES[class_id], 100 * probability))

